### launching of SPs



In [1]:
from datetime import timedelta
import pandas as pd

import pyodbc
from sql.config import SQL_SERVER, DB_NAME
import sql.naming_convention as nc
from sql.sql_requests import SQL_Communicator

In [2]:
def format_time_delta(duration: timedelta):
    # Get the total seconds
    total_seconds = duration.total_seconds()
    # days = duration.days
    hours = duration.seconds // 3600  # Convert seconds to hours
    minutes = (duration.seconds // 60) % 60  # Convert remaining seconds to minutes
    seconds = duration.seconds % 60
    if hours > 0:
        return f'{hours:02d}:{minutes:02d}:{int(seconds):02d}'
    if minutes > 0:
        return f'{minutes:02d}:{int(seconds):02d}'
    return f'{total_seconds:4.2f}'

In [3]:
entity_names =  ['Symbol_IBESTickerEnhanced',
        'Symbol_TickerEnhanced',
        'Symbol_CusipEnhanced',
        'Symbol_FTCusip6Symbol',
        'Symbol_FTCusip9Symbol',
        'Symbol_FTISINSymbol',
        'Symbol_ISINEnhanced',
        'Symbol_SECCIKEnhanced',
        'Symbol_SECCIKSymbol',
        'Symbol_SedolEnhanced',
        'Symbol_FTSEDOLSymbol',]

durations = []
with SQL_Communicator() as sc:
    for en in entity_names:
        print(f'flow for target entity {en}..')
        en2 = nc.default_rename(en)
        sp_names = [nc.pull_sp_name(en2), nc.merge_sp_name(en2)]
        ent_durs, flow_td = [en2], timedelta()  
        for sp_name in sp_names:
            exec_stm = f"EXEC {sp_name}"
            print(f'- {exec_stm}...')
            td = sc.get_execution_metrics(exec_stm)
            ent_durs.append(format_time_delta(td))
            flow_td += td
        ent_durs.append(format_time_delta(flow_td))
        durations.append(ent_durs)
        # ent_durs.append(sum(ent_durs))
columns = ['Entity name', 'Pull SP', 'Merge SP',  'sum']
df = pd.DataFrame(durations, columns = columns)


connecting...
Connection successful!
flow for target entity Symbol_IBESTickerEnhanced..
- EXEC dbo.PullData_Symbol_IBESTickerEnhanced2_prc...
- EXEC dbo.MergeData_Symbol_IBESTickerEnhanced2_prc...
['Symbol_IBESTickerEnhanced2', '0.41', '1.07', '1.48']
flow for target entity Symbol_TickerEnhanced..
- EXEC dbo.PullData_Symbol_TickerEnhanced2_prc...
- EXEC dbo.MergeData_Symbol_TickerEnhanced2_prc...


In [7]:
df

,Entity name,Pull SP,Merge SP,sum
0,Symbol_FTCusip6Symbol2,1.69,2.94,4.63
